<a href="https://colab.research.google.com/github/UV1L/MyFirstNeutralNetwork/blob/main/NeutralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Моя первая нейронка](https://)**

Для начала загрузим токен для работы с API Kaggle:

In [ ]:
from google.colab import files


token = files.upload()

!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


Скачаем данные с соревнования https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

test_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train.csv               
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                
Archive:  test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_labels.csv         


Вот пример входных данных для теста:

In [ ]:
!head -10 train.csv

"id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"
"0000997932d777bf","Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
"000103f0d9cfb60f","D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
"000113f07ec002fd","Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
"0001b41b1c6bb37e","""
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents""""  -I think the refere

Запишем данные из csv в переменную

In [ ]:
import pandas as pd


csv_train_data = pd.read_csv("train.csv")
comments = csv_train_data["comment_text"]
y_train = csv_train_data[["toxic",
                          "severe_toxic",
                          "obscene",
                          "threat",
                          "insult",
                          "identity_hate"
                          ]]

Создаем токенизатор и обучаем его

Максимальное число слов = 10000

In [ ]:
from tensorflow.keras.preprocessing.text import *


my_tokenizer = Tokenizer(num_words=10000)

Обучаем токенизатор на тестовых комментариях, которые мы до этого получили

In [ ]:
my_tokenizer.fit_on_texts(comments)
my_tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'have': 18,
 'are': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'wikipedia': 28,
 'my': 29,
 'an': 30,
 'from': 31,
 'by': 32,
 'do': 33,
 'at': 34,
 'me': 35,
 'about': 36,
 'so': 37,
 'talk': 38,
 'what': 39,
 'can': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'will': 44,
 'please': 45,
 'no': 46,
 'would': 47,
 'one': 48,
 'like': 49,
 'just': 50,
 'they': 51,
 'he': 52,
 'which': 53,
 'any': 54,
 'been': 55,
 'should': 56,
 'more': 57,
 'we': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'here': 63,
 'see': 64,
 'also': 65,
 'his': 66,
 'think': 67,
 'because': 68,
 'know': 69,
 'how': 70,
 'edit': 71,
 'am': 72,
 "i'm": 73,
 'people': 74,
 'why': 75,
 'up': 76,
 'only': 77,
 "it's": 78,
 'out': 79,
 'articles': 80,
 'use': 81,

Получаем число символов, соотвествующих каждому слову в тестовых комментариях

In [ ]:
sequences = my_tokenizer.texts_to_sequences(comments)

Вот пример одного такого комментария:

In [ ]:
sequences[1]

[52,
 2635,
 13,
 555,
 3809,
 73,
 4556,
 2706,
 21,
 94,
 38,
 803,
 2679,
 992,
 589,
 8377,
 182]

Получаем численное представление тестовых данных с максимальной длиной комментария в 50 символов

In [ ]:
from keras.preprocessing.sequence import *


x_train = pad_sequences(sequences, 50)

In [ ]:
x_train[::]

array([[   0,    0,    0, ..., 4583, 2273,  985],
       [   0,    0,    0, ...,  589, 8377,  182],
       [   0,    0,    0, ...,    1,  737,  468],
       ...,
       [   0,    0,    0, ..., 8167, 3509, 4528],
       [   0,    0,    0, ...,  151,   34,   11],
       [   0,    0,    0, ..., 1627, 2056,   88]], dtype=int32)

Далее создаем модель последоваельной сети lstm

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SpatialDropout1D
from keras.layers import LSTM
from keras.datasets import imdb

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(10000, 128, input_length=50))
model.add(SpatialDropout1D(0.5))

1.   Добавляем слой долго-краткосрочной памяти
2.   Добавляем полносвязный слой из 1 элемента для классификации, в качестве функции активации будем использовать сигмоидальную функцию
3.   Компилируем модель



Компилируем

In [ ]:
model.add(LSTM(40, return_sequences=True))
model.add(LSTM(40))
model.add(Dense(6, activation="sigmoid"))
model.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

Делаем чекпоинт c флагом save_best_only для сохранения самой качественной эпохи

In [ ]:
from keras.callbacks import *


h5 = "model.h5"
model_checkpoint = ModelCheckpoint(h5,
                                   monitor="val_accuracy",
                                   save_best_only=True,
                                   verbose=0.2)


Обучаем нейронную сеть (данные для обучения, ответы к данным для обучения, число эпох обучения, количество рецензий после анализа которого будут изменены веса,
куда записывать колбек)

In [ ]:
model.fit(x_train,
          y_train,
          epochs=7,
          batch_size=256,
          validation_split=0.2,
          callbacks=[model_checkpoint])

Epoch 1/7
499/499 [==============================] - 18s 30ms/step - loss: 0.2101 - accuracy: 0.7426 - val_loss: 0.0643 - val_accuracy: 0.9935

Epoch 00001: val_accuracy improved from -inf to 0.99351, saving model to model.h5
Epoch 2/7
499/499 [==============================] - 13s 26ms/step - loss: 0.0589 - accuracy: 0.9942 - val_loss: 0.0568 - val_accuracy: 0.9941

Epoch 00002: val_accuracy improved from 0.99351 to 0.99411, saving model to model.h5
Epoch 3/7
499/499 [==============================] - 12s 25ms/step - loss: 0.0520 - accuracy: 0.9940 - val_loss: 0.0541 - val_accuracy: 0.9941

Epoch 00003: val_accuracy did not improve from 0.99411
Epoch 4/7
499/499 [==============================] - 12s 24ms/step - loss: 0.0486 - accuracy: 0.9942 - val_loss: 0.0544 - val_accuracy: 0.9941

Epoch 00004: val_accuracy did not improve from 0.99411
Epoch 5/7
499/499 [==============================] - 12s 25ms/step - loss: 0.0459 - accuracy: 0.9943 - val_loss: 0.0542 - val_accuracy: 0.9940

Epo

Загрузим тестовые комментарии и объединим их с таблицей ответов по id, далее оставим только те, где столбец toxic != -1 

In [ ]:
comments_test = pd.read_csv("test.csv")
comments_labels = pd.read_csv("test_labels.csv")

comments_test_with_labels = pd.merge(comments_test, comments_labels, on="id")
toxic = comments_test_with_labels["toxic"]
comments_test_without_toxic = comments_test_with_labels[toxic != -1]

Преобразуем в числовое представление

In [ ]:
sequences_test = my_tokenizer.texts_to_sequences(comments_test_without_toxic["comment_text"])

x_test = pad_sequences(sequences_test, maxlen=50)
y_test = comments_test_without_toxic[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

Протестируем нейросеть:

In [ ]:
model.load_weights(h5)
model.evaluate(x_test, y_test, verbose=0.2)

[0.08075769245624542, 0.9976085424423218]

In [ ]:
my_test_non_toxic_comment = "Hello world!"
my_test_toxic_comment = "I will kill u"
first_sequence = my_tokenizer.texts_to_sequences([my_test_non_toxic_comment, my_test_toxic_comment])
pad_seq = pad_sequences(first_sequence, maxlen=50)
result = model.predict(pad_seq)
result

array([[0.25482798, 0.00238616, 0.04821115, 0.00354104, 0.06246819,
        0.01111174],
       [0.9054299 , 0.04156527, 0.60767466, 0.01921766, 0.6072712 ,
        0.05967374]], dtype=float32)